In this project we will be analyzing data from a big online store. Together with the marketing department, we've compiled a list of hypotheses that may help boost revenue.
We need to prioritize these hypotheses, launch an A/B test, and analyze the results.

The main goals of this analysis are:
1. To optimize the priorites of hypotheses.
2. Analyze and carry out recommendations as to stop or to continue the test.

In [1]:
# Import all libraries
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
from scipy import stats
import statsmodels.api as sm
# Config
import plotly.io as pio
pio.renderers.default = "png"

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Load the data
try:
    df = pd.read_csv('/content/drive/MyDrive/csv_fies/hypotheses_us.csv', sep=';')
    visits = pd.read_csv('/content/drive/MyDrive/csv_fies/visits_us.csv')
    orders = pd.read_csv('/content/drive/MyDrive/csv_fies/orders_us.csv')
except:
    df = pd.read_csv('/datasets/hypotheses_us.csv', sep=';')
    visits = pd.read_csv('/datasets/visits_us.csv')
    orders = pd.read_csv('/datasets/orders_us.csv')    

## PART 1. PREPROCESSING

In [3]:
# Store the datasets in a list
datasets = [df, visits, orders]

In [4]:
# Customized function for data exploring
def explore(data):
  print('*_*' * 20)
  print()
  print('Dataframe Info:')
  data.info()
  print()
  print('Missing values:')
  print(data.isnull().sum())
  print()
  print('Checking duplicates:')
  print(data.duplicated().sum())
  print()

In [5]:
for i in datasets:
  explore(i)

*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Effort      9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes

Missing values:
Hypothesis    0
Reach         0
Impact        0
Confidence    0
Effort        0
dtype: int64

Checking duplicates:
0

*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    62 non-null     object
 1   group   62 non-null     object
 2   visits  62 non-null    

We see our date columns are stored in an incorrect data type - we need to fix them, before we begin working with our data.

In [6]:
visits['date'] = visits['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
orders['date'] = orders['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

Lets' check the result and if our dates in **orders** and **visits** datasets match.

In [7]:
print('Visits table first date: {}, last date: {}'.format(visits['date'].min(), visits['date'].max()))
print('Orders table first date: {}, last date: {}'.format(orders['date'].min(), orders['date'].max()))

Visits table first date: 2019-08-01 00:00:00, last date: 2019-08-31 00:00:00
Orders table first date: 2019-08-01 00:00:00, last date: 2019-08-31 00:00:00


We have our dates range in a month: from August 01, 2019 till August 31, 2019.

The `df` dataframe stores the descriptions of all the hypothesis we have. Lets' have a closer look at them.

In [8]:
pd.set_option('display.max_colwidth', 1)
df

,Hypothesis,Reach,Impact,Confidence,Effort
0,Add two new channels for attracting traffic. This will bring 30% more users,3,10,8,6
1,Launch your own delivery service. This will shorten delivery time,2,5,4,10
2,Add product recommendation blocks to the store's site. This will increase conversion and average purchase size,8,3,7,3
3,Change the category structure. This will increase conversion since users will find the products they want more quickly,8,3,3,8
4,Change the background color on the main page. This will increase user engagement,3,1,1,1
5,Add a customer review page. This will increase the number of orders,3,2,2,3
6,Show banners with current offers and sales on the main page. This will boost conversion,5,3,8,3
7,Add a subscription form to all the main pages. This will help you compile a mailing list,10,7,8,5
8,Launch a promotion that gives users discounts on their birthdays,1,9,9,5


There might be mistakes in our datasets; for example, some of the visitors might have gotten into both groups -  A and B.

In [9]:
# Group tha data by user id and count unique values of groups
duplicates = orders.groupby('visitorId')['group'].nunique().sort_values(ascending=False)
duplicates

visitorId
2654030115    2
3891541246    2
3941795274    2
1404934699    2
351125977     2
             ..
1476256851    1
1476741193    1
1484903144    1
1485964287    1
4283872382    1
Name: group, Length: 1031, dtype: int64

Indeed, we see that some users have more than 1 unique group, which contradicts the idea of a valid A/B test, we need to filter out the users, that had more than 1 group out of our samples.

In [10]:
# Store the id's of duplicated gropus users in a separate list
duplicates = duplicates[duplicates > 1].index.to_list()

In [11]:
# Eliminate the duplicated id's out of the orders table
orders = orders[np.logical_not(orders['visitorId'].isin(duplicates))]

In [12]:
orders.shape

(1016, 5)

Previously the orders table contained 1197 rows, we got rid of the duplicates and left with the filtered data.

### Intermediate conclusion 


---
The datasets and columns we have in our project:
1. `Hypothesis` dataset with columns
 - `Hypotheses` — brief descriptions of the hypotheses
 - `Reach` — user reach, on a scale of one to ten
 - `Impact` — impact on users, on a scale of one to ten
 - `Confidence` — confidence in the hypothesis, on a scale of one to ten
 - `Effort` — the resources required to test a hypothesis, on a scale of one to ten. The higher the Effort value, the more resource-intensive the test.
2. `Orders` dataset with columns
 - `transactionId` — order identifier
 - `visitorId` — identifier of the user who placed the order
 - `date` — date of the order
 - `revenue` — from the order
3. `Visits` dataset with columns 
 - `date` — date
 - `group` — A/B test group
 - `visits` — the number of visits on the date specified in the A/B test group specified


We explored our datasets, found no missing values. We located user ID's with relation to both A and B test groups, which we omitted. We fixed the data types in the date coulmns. We can start working with our data.

## PART 2. PRIORITIZING HYPOTHESIS


---


### **Apply the ICE framework to prioritize hypotheses. Sort them in descending order of priority**. 

The ICE scoring is one of the most popular frameworks for hypothesis prioriityzing. It takes in cout such parameters as:

`IMPACT` - it is the level of how strongly the change affects' the useers and the given metric

`CONFIDENCE` - the more exact the amount of usres afected by the change; the more exact we can predict the impact; the more preciesly we ca calculate the effort - the higher the confidence score will be.

`EFFORT` - the amount of resources needed to complete the test - manpower and time resources.

The ICE score will be calculated thus:


> ICE Score = Impact x Confidence x Effort.





In [13]:
df['ICE'] = df['Impact'] * df['Confidence'] * df['Effort']
df.sort_values(by = 'ICE', ascending=False)

,Hypothesis,Reach,Impact,Confidence,Effort,ICE
0,Add two new channels for attracting traffic. This will bring 30% more users,3,10,8,6,480
8,Launch a promotion that gives users discounts on their birthdays,1,9,9,5,405
7,Add a subscription form to all the main pages. This will help you compile a mailing list,10,7,8,5,280
1,Launch your own delivery service. This will shorten delivery time,2,5,4,10,200
3,Change the category structure. This will increase conversion since users will find the products they want more quickly,8,3,3,8,72
6,Show banners with current offers and sales on the main page. This will boost conversion,5,3,8,3,72
2,Add product recommendation blocks to the store's site. This will increase conversion and average purchase size,8,3,7,3,63
5,Add a customer review page. This will increase the number of orders,3,2,2,3,12
4,Change the background color on the main page. This will increase user engagement,3,1,1,1,1


### **Apply the RICE framework to prioritize hypotheses. Sort them in descending order of priority.**

The RICE framework is a similar but a more precise methodology of priorityzing working hypothesis. The components are the same as with ICE, with an addition of taking in count the `REACH` parameter - it is the number of users that will be exposed to the change.

The RICE score formula is:

> RICE Score = (Reach x Impact x Confidence) / Effort.

In [14]:
df['RICE'] = (df['Reach'] * df['Impact'] * df['Confidence']) / df['Effort']
df.sort_values(by = 'RICE', ascending=False)

,Hypothesis,Reach,Impact,Confidence,Effort,ICE,RICE
7,Add a subscription form to all the main pages. This will help you compile a mailing list,10,7,8,5,280,112.0
2,Add product recommendation blocks to the store's site. This will increase conversion and average purchase size,8,3,7,3,63,56.0
0,Add two new channels for attracting traffic. This will bring 30% more users,3,10,8,6,480,40.0
6,Show banners with current offers and sales on the main page. This will boost conversion,5,3,8,3,72,40.0
8,Launch a promotion that gives users discounts on their birthdays,1,9,9,5,405,16.2
3,Change the category structure. This will increase conversion since users will find the products they want more quickly,8,3,3,8,72,9.0
1,Launch your own delivery service. This will shorten delivery time,2,5,4,10,200,4.0
5,Add a customer review page. This will increase the number of orders,3,2,2,3,12,4.0
4,Change the background color on the main page. This will increase user engagement,3,1,1,1,1,3.0


In [15]:
fig = px.scatter(x=df.index, y = df['RICE'])
fig.update_xaxes(title_text = 'Hypotheses number')
fig.update_yaxes(title_text = 'RICE score')
fig.show('colab')

### **Show how the prioritization of hypotheses changes when you use RICE instead of ICE. Provide an explanation for the changes.**


---

As shown in the tables above, when we are guided by two of our given frameworks the priority of the hypothesis changes, for example: 

According to **ICE** our top 3 priority hypothesis are **№ 0, 8 and 7**, but when in addition to ICE we take in account a very important parameter such as the REACH we get different and a more precise result. 

Note how the 7-th hypothesis according to **RICE** is the highest in priority because it has the highest `REACH` score, but if we comapre the same hypothesis' position in the **ICE** score column it is lower in priority because all the other parameters - impact, confidence and effort - are not high enough in comparison with other hypothesis. And it makes sence: hypothesis №7 suggests that we apply a change to all the main pages, so really every user will be exposed to the change. 

Thus, we can conclude, that according to **RICE score** the top 3 priority hypothesis should be tested are **№7, 2 and 0** respectively.

##  PART 3. A/B TEST ANALYSIS


---




### **Graph cumulative revenue by group. Make conclusions and conjectures.**

For each one of the groups we will pair a match of each date of the given period - from August 01 2019 till the August 31-st 2019 in our case.

In [16]:
# Choose the unique date and group notations
dates_for_groups = orders[['date','group']].drop_duplicates()
dates_for_groups

,date,group
0,2019-08-15,B
7,2019-08-15,A
45,2019-08-16,A
47,2019-08-16,B
55,2019-08-01,A
...,...,...
1068,2019-08-12,A
1114,2019-08-13,B
1116,2019-08-13,A
1149,2019-08-14,B


Having those dates gathered for each group we will aggregate our data in the `orders` and `visits` datasets, selecting each group and date to accumulate all the observations to one row with one date and one group for each metric we need: amount of orders, buyers, visitors, total revenue.

In [17]:
orders_agg = dates_for_groups.apply(
    lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])]
    .agg({'date' : 'max', 
          'group' : 'max', 
          'transactionId' : pd.Series.nunique, 
          'visitorId' : pd.Series.nunique, 
          'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])
orders_agg

,date,group,transactionId,visitorId,revenue
55,2019-08-01,A,23,19,2266.6
66,2019-08-01,B,17,17,967.2
175,2019-08-02,A,42,36,3734.9
173,2019-08-02,B,40,39,3535.3
291,2019-08-03,A,66,60,5550.1
...,...,...,...,...,...
533,2019-08-29,B,510,490,74576.7
757,2019-08-30,A,460,437,52363.7
690,2019-08-30,B,531,511,77863.5
958,2019-08-31,A,468,445,53212.0


In [18]:
visits_agg = dates_for_groups.apply(
    lambda x: visits[np.logical_and(visits['date'] <= x['date'], visits['group'] == x['group'])]
    .agg({'date': 'max',
          'group': 'max',
          'visits': 'sum'}), axis=1).sort_values(by=['date', 'group'])
visits_agg

,date,group,visits
55,2019-08-01,A,719
66,2019-08-01,B,713
175,2019-08-02,A,1338
173,2019-08-02,B,1294
291,2019-08-03,A,1845
...,...,...,...
533,2019-08-29,B,17708
757,2019-08-30,A,18037
690,2019-08-30,B,18198
958,2019-08-31,A,18736


In [19]:
# Merge the aggreagted data to one table
data_cumulation = orders_agg.merge(visits_agg, left_on=['date', 'group'], right_on=['date', 'group'])

In [20]:
# Rename the columns
data_cumulation.columns = ['date', 'group', 'n_transactions', 'n_visitors', 'revenue', 'visits']

In [21]:
data_cumulation.head()

,date,group,n_transactions,n_visitors,revenue,visits
0,2019-08-01,A,23,19,2266.6,719
1,2019-08-01,B,17,17,967.2,713
2,2019-08-02,A,42,36,3734.9,1338
3,2019-08-02,B,40,39,3535.3,1294
4,2019-08-03,A,66,60,5550.1,1845


Now that we cumulated the data to one table we can plot our revenue dynamics by group.

In [22]:
fig = px.line(data_cumulation, x='date', y='revenue', color='group', markers=True, 
              title = 'Cumulative revenue by group',
              labels=dict(revenue='Revenue', date='Dates'))
fig.show()

As the graph shows - both, A and B groups start with a close level of revenue untill a breakpoint that happened on the 18-th of August when B group has a sudden spike up, but both groups' revenues increase evenly (parallel).

### **Graph cumulative average order size by group. Make conclusions and conjectures.**

In [23]:
# Calculate the average order size to a new column
data_cumulation['avg_order_size'] = data_cumulation['revenue'] / data_cumulation['n_transactions']
data_cumulation

,date,group,n_transactions,n_visitors,revenue,visits,avg_order_size
0,2019-08-01,A,23,19,2266.6,719,98.547826
1,2019-08-01,B,17,17,967.2,713,56.894118
2,2019-08-02,A,42,36,3734.9,1338,88.926190
3,2019-08-02,B,40,39,3535.3,1294,88.382500
4,2019-08-03,A,66,60,5550.1,1845,84.092424
...,...,...,...,...,...,...,...
57,2019-08-29,B,510,490,74576.7,17708,146.228824
58,2019-08-30,A,460,437,52363.7,18037,113.834130
59,2019-08-30,B,531,511,77863.5,18198,146.635593
60,2019-08-31,A,468,445,53212.0,18736,113.700855


In [24]:
fig = px.line(data_cumulation, x='date', y='avg_order_size', color='group', markers=True, 
              title='Cumulative metric of average order size by group',
              labels=dict(avg_order_size='Average order size', date='Dates'))
fig.show()

On the graph we see that group A started with a higher average check, than B, there are some fluctuations at the beginning - there are rises and drops for each group, but again as with the previous chart there is a breakpoint on August, 18 2019 when group B increases significantly in comparison to A group and the metrics are stabilizing.

### **Graph the relative difference in cumulative average order size for group B compared with group A. Make conclusions and conjectures.**

In [25]:
# Filter the data by group attribute
groupA = data_cumulation[data_cumulation['group'] == 'A']
groupB = data_cumulation[data_cumulation['group'] == 'B']

In [26]:
groupB.head()

,date,group,n_transactions,n_visitors,revenue,visits,avg_order_size
1,2019-08-01,B,17,17,967.2,713,56.894118
3,2019-08-02,B,40,39,3535.3,1294,88.382500
5,2019-08-03,B,54,53,4606.9,1803,85.312963
7,2019-08-04,B,68,66,6138.5,2573,90.272059
9,2019-08-05,B,89,87,7587.8,3280,85.256180


Now we can merge the data back together with a distinction of columns for metrics for groups A and B separatly.

In [27]:
AB_revenue_table = groupA.merge(groupB, how='left', left_on='date', right_on='date', suffixes=['A', 'B'])
AB_revenue_table.shape

(31, 13)

In [28]:
fig = px.line(AB_revenue_table, x='date', y=AB_revenue_table['avg_order_sizeB'] / AB_revenue_table['avg_order_sizeA'] - 1, markers=True, 
              title = 'Relative difference in cumulative average order size for group B compared with group A')
fig.update_xaxes(title_text='Dates')
fig.update_yaxes(title_text='Relative difference')
fig.add_hline(y=0, line_dash='dash')
fig.show()

The further the points on the graph are located from the 0 dash line the bigger the relative difference between average order sizes in groups A and B. The spike in the same date as we saw before can mean that there is a certain anomaly. The metric seems stabilizing after the spike on August 18-th.

### **Calculate each group's conversion rate as the ratio of orders to the number of visits for each day. Plot the daily conversion rates of the two groups and describe the difference. Draw conclusions and make conjectures.**

In [29]:
# Calculate the conversion rate to a separate column
data_cumulation['conversion'] = data_cumulation['n_transactions'] / data_cumulation['visits']
data_cumulation.head()

,date,group,n_transactions,n_visitors,revenue,visits,avg_order_size,conversion
0,2019-08-01,A,23,19,2266.6,719,98.547826,0.031989
1,2019-08-01,B,17,17,967.2,713,56.894118,0.023843
2,2019-08-02,A,42,36,3734.9,1338,88.926190,0.031390
3,2019-08-02,B,40,39,3535.3,1294,88.382500,0.030912
4,2019-08-03,A,66,60,5550.1,1845,84.092424,0.035772


In [30]:
fig = px.line(data_cumulation, x='date', y='conversion', color='group', markers=True, 
              title = 'Conversion rate dynamics',
              labels=dict(date='Dates', conversion='Conversion to purchase'))
fig.show()

Group B started with a comparatively lower conversion rate than group A, but it increased around August 6-th and shows a steadily higher conversion. Both lines look like they are stabilizing. Lets' have a look at their relative difference.

In [31]:
fig = px.line(AB_revenue_table, y=(
    AB_revenue_table['n_transactionsB'] / AB_revenue_table['visitsB']) / (AB_revenue_table['n_transactionsA'] / AB_revenue_table['visitsA']) - 1, 
    x='date', title='Relative difference in the cumulative conversion of group B to group A')
fig.add_hline(y=0, line_dash='dash')
fig.add_hline(y=-0.1, line_dash='dash')
fig.update_xaxes(title_text='Dates')
fig.update_yaxes(title_text='Relative difference')
fig.show()

Group B conversion is consistently better than of group A by aproximatly 15% after August, 10-th from the start of the test.

### **Plot a scatter chart of the number of orders per user. Make conclusions and conjectures.**

In [32]:
orders_by_users = orders.groupby('visitorId', as_index = False).agg({'transactionId':'nunique'}).sort_values(by='transactionId', ascending = False)
orders_by_users.columns = ['visitorId', 'orders']
orders_by_users.head()

,visitorId,orders
908,3967698036,3
55,249864742,3
478,2108163459,3
687,2988190573,3
890,3908431265,3


In [33]:
fig = px.histogram(orders_by_users['orders'], title='Histogram of number of orders per user')
fig.update_xaxes(title_text='Number of orders')
fig.show()

Most of our users make 1 order. Let's plot those values on a scatterplot.

In [34]:
x_values = pd.Series(range(0,len(orders_by_users)))

In [35]:
fig = px.scatter(orders_by_users, x=x_values, y='orders', 
                 title='Scatterplot of number of orders per user')
fig.update_xaxes(title_text='Number of orders')
fig.show()

Summing up the plots, we can definitely say that the majority of our users make no more than 2 orders.

### **Calculate the 95th and 99th percentiles for the number of orders per user. Define the point at which a data point becomes an anomaly.**


In [36]:
print(np.percentile(orders_by_users['orders'], [90, 95, 99])) 

[1. 1. 2.]


Only 1% of our users would make more than 2 orders, 10% of users would make more than 1 order. We will take the 99-th percentile as the higher border of the confidence interval, all orders above 2 may be considered anomalies.

### **Plot a scatter chart of order prices. Make conclusions and conjectures.**

In [37]:
orders['revenue'].describe()

count    1016.000000 
mean     130.770866  
std      648.079810  
min      5.000000    
25%      20.775000   
50%      50.100000   
75%      126.950000  
max      19920.400000
Name: revenue, dtype: float64

This method shows us a quick lookup at how the revenue distributed in quartiles range: 25% of the orders costed between 5 and 20, half of the revenue lies below 50 and 75% orders revenue lies below 126. We can note that the gap between the 75% quartile and the max value revenue is really big. Lets' see what the histogram can show us.

In [38]:
fig = px.histogram(orders['revenue'], title='Histogram of revenue distribution per user')
fig.update_xaxes(title_text='Revenue per user')
fig.show()

The histogram decreases at approximately 600 - 700. This means most of our orders cost in this range, but there are a lot of users that made orders for a much higher price. Lets' try to locate them more precisely.

In [39]:
orders

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B
5,182168103,935554773,2019-08-15,35.0,B
...,...,...,...,...,...
1191,3592955527,608641596,2019-08-14,255.7,B
1192,2662137336,3733762160,2019-08-14,100.8,B
1193,2203539145,370388673,2019-08-14,50.1,A
1194,1807773912,573423106,2019-08-14,165.3,A


In [40]:
fig = px.scatter(x = orders.index, y = orders['revenue'], 
                 color = orders['group'],
                 title = 'Scatterplot of revenue distributon per user')
fig.update_xaxes(title_text='count')
fig.update_yaxes(title_text='Revenue per user')
fig.show()

Just as we saw before there are some abnormally big orders.

### **Calculate the 95th and 99th percentiles of order prices. Define the point at which a data point becomes an anomaly.**

In [41]:
print(np.percentile(orders['revenue'], [90, 95, 99])) 

[280.8   414.275 830.3  ]


About 5% of our orders cost higher than 414.275 and about 1% cost higher than 830.30. We will use the 95% percentile as the higher border.

### **Find the statistical significance of the difference in conversion between the groups using the raw data. Make conclusions and conjectures.**

For this part of our analysis we will need to calculate the number of visits in each group as the number of trials, and the number of orders as the number of successes of those trials. We will have two proportions of successes to trials for each group, which will eventually allow us to perform a **z-proportion test**.  

In [42]:
# Calculate the number of orders per each user for both groups
orders_by_usersA = orders[orders['group']=='A'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
orders_by_usersA.columns = ['visitorId', 'orders']

In [43]:
orders_by_usersB = orders[orders['group']=='B'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
orders_by_usersB.columns = ['visitorId', 'orders']

In [44]:
orders_by_usersA['orders']

0      1
1      1
2      1
3      1
4      1
      ..
440    1
441    1
442    1
443    1
444    1
Name: orders, Length: 445, dtype: int64

Now that we have both of our datasets ready we will use the z-proportion statistical test. 

Lets' formulate the null hypothesis:
> The proportions in group A and group B are equal

Thus, the alternative hypothesis for our test states:

> The proportions in group A and group B are NOT equal to each other

In [45]:
# Define the level of significance
alpha = 0.05

In [46]:
print('Number of orders in group А: {}'.format(orders_by_usersA['orders'].sum()))
print('Number of visits in group A: {}\n'.format(visits[visits['group']=='A']['visits'].sum()))
print('Number of orders in group B: {}'.format(orders_by_usersB['orders'].sum()))
print('Number of visits in group B: {}'.format(visits[visits['group']=='B']['visits'].sum()))

Number of orders in group А: 468
Number of visits in group A: 18736

Number of orders in group B: 548
Number of visits in group B: 18916


In [47]:
# Store the number of successes
s1 = orders_by_usersA['orders'].sum()
s2 = orders_by_usersB['orders'].sum()
# Store the number of trials
t1 = visits[visits['group']=='A']['visits'].sum()
t2 = visits[visits['group']=='B']['visits'].sum()

In [48]:
# Perform the test
z_label, p_value = sm.stats.proportions_ztest([s1, s2], [t1, t2])

In [49]:
p_value

0.01685138043470311

In [50]:
if (p_value < alpha):
    print("Rejecting the null hypothesis: there is a significant difference between the proportions")
else:
    print("Failed to reject the null hypothesis: there is no reason to consider the proportions different") 

Rejecting the null hypothesis: there is a significant difference between the proportions


The conversions in group A and group B are not the same.

### **Find the statistical significance of the difference in average order size between the groups using the raw data. Make conclusions and conjectures.**

In [51]:
# Create a sample with 1s' and 0s' that represent if user made purchase or not
sampleA = pd.concat([orders_by_usersA['orders'], 
                     pd.Series(0, index=np.arange(visits[visits['group']=='A']['visits'].sum() - len(orders_by_usersA['orders'])), 
                               name='orders')], axis=0)

In [52]:
 # Repeat the process for group B
sampleB = pd.concat([orders_by_usersB['orders'], 
                      pd.Series(0, index=np.arange(visits[visits['group']=='B']['visits'].sum() - len(orders_by_usersA['orders'])), 
                                name='orders')], axis=0) 

In [53]:
# Extract all revenues for groups A and B
avg_sampleA = orders.query('group == "A"')['revenue']
avg_sampleA

7       20.1 
8       215.9
10      65.7 
12      115.4
14      170.9
        ...  
1185    15.2 
1187    15.5 
1188    5.7  
1193    50.1 
1194    165.3
Name: revenue, Length: 468, dtype: float64

In [54]:
avg_sampleB = orders.query('group == "B"')['revenue']
avg_sampleB

0       30.4  
1       15.2  
3       155.1 
4       40.5  
5       35.0  
        ...   
1186    240.6 
1190    135.3 
1191    255.7 
1192    100.8 
1196    3120.1
Name: revenue, Length: 548, dtype: float64

We are checking the statistical significance of the difference in average order sizes between group A and group B. We need to apply the Mann–Whitney U criterion - it is a non-parametric statistical
criterion used to compare the level of difference of certain metrics in two
independent samples. As with any other statistical test we have to state the hypotheses first.
> The null hypothesis states that there is no difference in average order sizes between group A and group B:

$ H_{0}:$ $\bar{x}_{A} = \bar{x}_{B} $

Previously on one of our plots we saw that the average order sizes of group B consistently outperformed average order sizes in group A, taking that to account, 
>The alternative hypothesis will be that average order size in group A is lower than in group B:

$ H_{0}:$ $\bar{x}_{A} < \bar{x}_{B} $

If the p-value that we will get as the result of our test will be greater than 0.05 (the significance level) it would mean that there is no difference between the samples and we wouldn't be able to reject the null hypothesis, and vice versa: if the p-value will be lower than 0.05 that would indicate that average order size of group A is indeed lower than in group B.

In [55]:
print("The p-level of significance is {0:.3f}".format(stats.mannwhitneyu(avg_sampleA, avg_sampleB)[1]))

print("The relative loss/gain in group B is {0:.3f}".format(avg_sampleB.mean()/avg_sampleA.mean()-1)) 

The p-level of significance is 0.431
The relative loss/gain in group B is 0.278


In [56]:
if (stats.mannwhitneyu(avg_sampleA, avg_sampleB)[1] < alpha):
    print("Rejecting the null hypothesis: there is a significant difference between the average order sizes")
else:
    print("Failed to reject the null hypothesis: there is no reason to consider the average order sizes are different") 

Failed to reject the null hypothesis: there is no reason to consider the average order sizes are different


We can't reject the null hypothesis, the p-level is higher than 0.05, which means that with 95% confidence we can state that there is no difference between the average sizes of the orders between groups A and B. 

### **Find the statistical significance of the difference in conversion between the groups using the filtered data. Make conclusions and conjectures.**

Earlier we ran statistical tests on raw data and got some results. Previously in our research we stated that there are some anomalies in our data. Lets' filter them out and run the same tests on the outlier-free data in order to see how the anomalies affect the results.

In [57]:
# Define the outliers
order_limit = 2
revenue_limit = 415

In [58]:
# Filter out the outliers for both groups
orders_by_usersA[orders_by_usersA['orders'] > order_limit]['visitorId']

58     611059232 
211    2108163459
310    2988190573
409    3908431265
416    3967698036
Name: visitorId, dtype: int64

In [59]:
orders_by_usersB[orders_by_usersB['orders'] > order_limit]['visitorId']

33     249864742 
347    2742574263
Name: visitorId, dtype: int64

In [60]:
# Join all users with big amount of orders to one subset
anomal_orders = pd.concat([orders_by_usersA[orders_by_usersA['orders'] > order_limit]['visitorId'], orders_by_usersB[orders_by_usersB['orders'] > order_limit]['visitorId']], axis = 0)
anomal_orders

58     611059232 
211    2108163459
310    2988190573
409    3908431265
416    3967698036
33     249864742 
347    2742574263
Name: visitorId, dtype: int64

In addition to this list of user's ids' we need to find the user's id's with too expensive orders.

In [61]:
expensive_orders = orders[orders['revenue'] > revenue_limit]['visitorId']
expensive_orders

73      2947100995
77      787824685 
78      3288820196
131     2254586615
137     1211585711
173     2953427180
208     3118156181
273     2278620397
287     2254456485
339     834731816 
420     1606766439
421     1357170471
425     1920142716
481     3177790019
527     2837914161
530     2912540959
549     1367935136
551     1010866419
562     1452540367
563     935349363 
568     113298937 
570     2247666522
613     3931967268
648     382846391 
654     816783716 
684     358944393 
709     950626008 
723     4009017467
728     3357398384
743     4133034833
744     888512513 
746     3321013705
751     689129267 
755     2279926127
867     3372131337
890     2853587382
925     2840985286
928     204675465 
940     4003628586
949     887908475 
960     2999642019
1006    2261934787
1038    1605520752
1087    3590655705
1099    148427295 
1103    1164614297
1121    1604853044
1136    1307669133
1137    759473111 
1142    2606908679
1196    2108080724
Name: visitorId, dtype: int64

In [62]:
# Join the big and expensive orders to one dataframe
abnormal_users = pd.concat([anomal_orders, expensive_orders], axis = 0)
print('We manged to find {} abnormally behaved users.'.format(abnormal_users.shape[0]))

We manged to find 58 abnormally behaved users.


In [63]:
# Filter out the abnormaly behaved user ids'
orders_by_usersA_filt = orders_by_usersA[~(orders_by_usersA['visitorId'].isin(abnormal_users))]
orders_by_usersB_filt = orders_by_usersB[~(orders_by_usersB['visitorId'].isin(abnormal_users))]

As before we will be running the z-proportion statistical test. 

Lets' formulate the null hypothesis:
> The proportions in group A and group B are equal

Thus, the alternative hypothesis for our test states:

> The proportions in group A and group B are NOT equal to each other

Lets' define number of trials as the amount of all visits our users made and number of successes as the number of orders. 

In [64]:
print('Number of orders in group А: {}'.format(orders_by_usersA_filt['orders'].sum()))
print('Number of visits in group A: {}\n'.format(visits[visits['group']=='A']['visits'].sum()))
print('Number of orders in group B: {}'.format(orders_by_usersB_filt['orders'].sum()))
print('Number of visits in group B: {}'.format(visits[visits['group']=='B']['visits'].sum()))

Number of orders in group А: 430
Number of visits in group A: 18736

Number of orders in group B: 513
Number of visits in group B: 18916


In [65]:
sA = orders_by_usersA_filt['orders'].sum()
sB = orders_by_usersB_filt['orders'].sum()
tA = visits[visits['group']=='A']['visits'].sum()
tB = visits[visits['group']=='B']['visits'].sum()

In [66]:
z_label_, p_value_ = sm.stats.proportions_ztest([sA, sB], [tA, tB])

In [67]:
p_value_

0.00963392628173691

In [68]:
if (p_value_ < alpha):
    print("Rejecting the null hypothesis: there is a significant difference between the proportions")
else:
    print("Failed to reject the null hypothesis: there is no reason to consider the proportions different") 

Rejecting the null hypothesis: there is a significant difference between the proportions


### **Find the statistical significance of the difference in average order size between the groups using the filtered data. Make conclusions and conjectures.**

As with the raw data we will run the Mann-Whitney U test.

In [69]:
avg_samplA_filt = orders[np.logical_and(orders['group']=='A', np.logical_not(orders['visitorId'].isin(abnormal_users)))]['revenue']

In [70]:
avg_samplB_filt = orders[np.logical_and(orders['group']=='B', np.logical_not(orders['visitorId'].isin(abnormal_users)))]['revenue']

In [71]:
print("The p-level of significance is {0:.3f}".format(stats.mannwhitneyu(avg_samplA_filt, avg_samplB_filt)[1]))

print("The relative loss/gain in group B is {0:.3f}".format(avg_samplB_filt.mean()/avg_samplA_filt.mean()-1)) 

The p-level of significance is 0.340
The relative loss/gain in group B is -0.047


In [72]:
if (stats.mannwhitneyu(avg_samplA_filt, avg_samplB_filt)[1] < alpha):
    print("Rejecting the null hypothesis: there is a significant difference between the average order sizes")
else:
    print("Failed to reject the null hypothesis: there is no reason to consider the average order sizes are different") 

Failed to reject the null hypothesis: there is no reason to consider the average order sizes are different


### **Make a decision based on the test results. The possible decisions are:** 
1. Stop the test, consider one of the groups the leader. 
2. Stop the test, conclude that there is no difference between the groups. 
3. Continue the test.

## PART 4. GENERAL CONCLUSION


---





Lets' sum up the results of all our tests:

We checked 2 hypotheses, both of them were tested on  raw data and filtered data without previously located outliers. And here are the results we received:

**1. There is a statistical significance in the difference in conversion rates of group A and group B**
 - raw data showed - reject of the null hypothesis: there is a significant difference between the proportions, p-value = 0.01685138043470311
 - filtered data showed - reject the null hypothesis: there is a significant difference between the proportions, p-value = 0.00963392628173691
**2. There is a statistical significance in the difference in average order size between the groups**
 - raw data showed - failed to reject the null hypothesis: there was no reason to consider the average order sizes are different

The p-level of significance is 0.862 and the relative gain in group B is 27.8%
 - filtered data showed - failed to reject the null hypothesis: there is no reason to consider the average order sizes are different

The p-level of significance is 0.680 and the relative loss in group B is 4.7%

So we see that the results are the same with both data sets - raw and filtered, except one small, but very important detail: according to rhe raw data test average order size in group B did better and shows a gain of 27.8%. That shows 2 points: 
1. The outliers contribute to the average order size.
2. Group B succeeded in the test on average order size. Considering, that this metric is more reliable in terms of revenue, we would recommend to stop the test and consider group B the leader.

In this project we  researched data from a big online store.
Our main goals were :

**1. To optimize the priorities of hypotheses.**

**2. Analyze and carry out recommendations as to stop or to continue the test.**
 
**- In the PREPROCESSING part of the project**

We explored our data for errors, missing values and incorrect data types, we fixed the date types, and located an error in the data - some users belonged to both A and B groups and we omitted them.

**- In the PRIORITIZING  HYPOTHESES part**

We applied the `ICE` and `RICE` frameworks and saw that the `RICE` scoring approach was the more precise one, and thanks to it we established that the top 3 hypotheses are **№7, 2 and 0** respectively.

**- In the A/B TEST ANALYSIS part**

We did visualizations, such as:
 - *Plot of cumulative revenue by group:* the graph showed - both, A and B groups start with a close level of revenue until a breakpoint that happened on the 18-th of August when B group has a sudden spike up, but both groups' revenues increased evenly (parallel).
 - *Plot of cumulative average order size by group:* on this graph we saw that group A started with a higher average check, than B, there were some fluctuations at the beginning - there were rises and drops for each group, but again as with the previous chart there was a breakpoint on August, 18 2019 when group B increased significantly in comparison to A group and the metrics stabilized.
 - *Plot of the relative difference in cumulative average order size for group B compared with group A*: the further the points on the graph were located from the 0 dash line the bigger the relative difference between average order sizes in groups A and B. So they diverged to opposite directions almost until August 18-th and then there was the spike in the same date as we saw before, which happened probably due to a certain anomaly. 
 - *We calculated each group's conversion rate and plotted the daily conversion rates of the two groups*: group B started with a comparatively lower conversion rate than group A, but it increased around August 6-th and showed a steadily higher conversion rate. Both lines looked like they were stabilized. In the graph of relative difference of conversion rates we found that group B’s conversion is consistently better than that of group A by aproximatly 15% after August, 10-th from the start of the test.

After examining the data, we located outliers and surges that stood in our way, so we determined them as abnormally big orders = 1  and extremely high revenues = 415. 
 We ran several tests on raw and filtered data and found that **group B succeeded the test** in terms of average order size gain 27.8%.


